# Contacts-0b: toevoegen (insert) en veranderen (update)

In dit notebook gaan we in op het toevoegen en veranderen van documenten in de database.

**TODO**: meer opdrachten toevoegen

## Initialisaties

In [ ]:
import os
import re
import pandas as pd
import numpy as np
from IPython.core.display import display, HTML
import pymongo

pd.set_option('max_colwidth',160)

userline = !echo $USER
username = userline[0]
dbname = username + "-demodb"
print("Database name: " + dbname)

from pymongo import MongoClient
print('Mongo version', pymongo.__version__)
client = MongoClient('localhost', 27017)
db = client[dbname]
collection = db.contacts

collection.drop()
os.system('/usr/local/bin/mongoimport -d ' + dbname + ' -c contacts adressen.json')

## Insert: toevoegen van documenten

Met de `insert`-opdracht kunnen we documenten aan een collectie toevoegen.
Vaak gebruiken we `insert_one`, om een enkel document toe te voegen.

**Let op**: als je deze opdracht herhaalt, wordt er nog een kopie van hetzelfde document aan de collectie toegevoegd.

**Opdracht** Ga dit na.

In [ ]:
person = {"name": "Sylvia Hansma", 
          "email": "sylh123@hotmail.com", 
          "address": {"street": "Rijksstraatweg 84", "city": "Halfweg"}
         }
collection.insert_one(person)

In [ ]:
list(collection.find())

## Upsert: overschrijven of toevoegen

Als je een document wilt toevoegen, waarbij je een eventueel bestaand document wilt overschrijven, dan kun je een update doen met `upsert=True`.
Dit betekent dat je een bestaand document overschrijft (update), of, als dat er niet is, een nieuw document toevoegt (insert).

> Hier moet je ook voorzichtig mee zijn: er kunnen bijvoorbeeld meerdere personen met dezelfde naam zijn. 
  Je kunt het query-deel van bijvoorbeeld uitbreiden met het email-adres.

In [ ]:
person = {"name": "Sylvia Hansma", 
          "email": "sylh123@hotmail.com", 
          "address": {"street": "Rijksstraatweg 84", "city": "Halfweg"}
         }
collection.update_one({"name": person["name"]}, {"$set": person}, upsert=True)
list(collection.find())

## Update

Door middel van een update-opdracht kunnen we bepaalde velden veranderen of toevoegen (via `$set`, met een aantal nieuwe waarden).

In [ ]:
collection.update_one({"name": "Anna Verschuur"}, {"$set": {"isFamily": True, "email": "anna33@gmail.com"}})

obj = collection.find_one({"name": "Anna Verschuur"})
print(obj)

In [ ]:
upd_obj = {"address.street": "Mozartplein 73", "address.city": "Rotterdam", "address.postcode": "2020 BA"}

collection.update_one({"name": "Anna Verschuur"}, {"$set": upd_obj})
obj = collection.find_one({"name": "Anna Verschuur"})
print(obj)

### Unset

Je kunt een ongewenste eigenschap (property) verwijderen met de `$unset`-opdracht. (De waarde van de property is in dit geval niet relevant.)

> Deze `$unset` kan handig zijn als een foute update resulteert in het toevoegen van een nieuw verld in plaats van in een update van een bestaand veld.

In [ ]:
upd_obj = {"isFamily": True}
collection.update_one({"name": "Anna Verschuur"}, {"$unset": upd_obj})